In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

C:\Users\User\AppData\Local\Temp\ipykernel_25284\644766539.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("D://New folder//ML//Completed//Hitters.csv")
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    object 
dtypes: float64

In [4]:
df['Salary'].fillna(df['Salary'].median(skipna=True),inplace=True)
df.isna().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_25284\1218454702.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salary'].fillna(df['Salary'].median(skipna=True),inplace=True)


AtBat        0
Hits         0
HmRun        0
Runs         0
RBI          0
Walks        0
Years        0
CAtBat       0
CHits        0
CHmRun       0
CRuns        0
CRBI         0
CWalks       0
League       0
Division     0
PutOuts      0
Assists      0
Errors       0
Salary       0
NewLeague    0
dtype: int64

In [5]:
dms = pd.get_dummies(df[['League','Division','NewLeague']],drop_first=True)

In [6]:
y = df['Salary']
x_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')
x = pd.concat([x_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [8]:
lasso_model = Lasso().fit(x_train,y_train)
lasso_model.intercept_

C:\Users\User\anaconda3\envs\Research\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.923e+06, tolerance: 3.853e+03
  model = cd_fast.enet_coordinate_descent(


342.8733925858767

In [9]:
lasso_model.coef_

array([-1.98558949e+00,  5.50494749e+00,  4.79612807e+00,  1.02123896e-01,
       -8.11521080e-01,  4.87004116e+00, -9.97808288e+00, -2.19391227e-01,
        6.16237616e-01,  9.03214960e-03,  8.73990383e-01,  7.84172593e-01,
       -8.13423037e-01,  1.83989460e-01,  4.04846687e-01, -4.08650952e+00,
        2.67092023e+01, -1.11463261e+02, -0.00000000e+00])

* Last coefficient is almost penalized to zero.

In [10]:
y_pred = lasso_model.predict(x_test)
mean_squared_error(y_test,y_pred,squared=False)

345.6190692407428

In [11]:
r2_score(y_test,y_pred)

0.3657513009571691

In [12]:
lasso_cv_model = LassoCV(alphas=np.random.randint(0,1000,100), cv=10, max_iter=10000, n_jobs=-1).fit(x_train,y_train)

In [13]:
lasso_cv_model.alpha_

12

In [14]:
lasso_tuned = Lasso().set_params(alpha=12).fit(x_train,y_train)
y_pred_tuned = lasso_tuned.predict(x_test)
mean_squared_error(y_test,y_pred_tuned, squared=False)

C:\Users\User\anaconda3\envs\Research\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.157e+06, tolerance: 3.853e+03
  model = cd_fast.enet_coordinate_descent(


346.31258364269667

In [15]:
pd.Series(lasso_tuned.coef_, index=x_train.columns)

AtBat          -1.933168
Hits            5.153563
HmRun           2.560394
Runs            0.284007
RBI            -0.000000
Walks           4.829895
Years          -4.953629
CAtBat         -0.255285
CHits           0.720859
CHmRun          0.138474
CRuns           0.855697
CRBI            0.727731
CWalks         -0.791287
PutOuts         0.179582
Assists         0.385342
Errors         -2.811569
League_N        0.000000
Division_W    -64.256747
NewLeague_N     0.000000
dtype: float64